In [79]:
import yfinance as yf
import pandas as pd
import datetime
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np

# 📌 Symbol for NIFTY 50 on Yahoo Finance
nifty_symbol = "^NSEI"

# ⏳ 20 years from today 😃
# You can customize start & end dates
start_date =  datetime.datetime.now() - datetime.timedelta(days=800)
end_date = datetime.datetime.now()

# Download data
nifty_data = yf.download(nifty_symbol, start=start_date, end=end_date, progress=False)
nifty_data.columns = nifty_data.columns.droplevel(1)
nifty_data.tail()


C:\Users\Saikat\AppData\Local\Temp\ipykernel_1528\4235623140.py:18: FutureWarning:

YF.download() has changed argument auto_adjust default to True



Price,Close,High,Low,Open,Volume
Date,,,,,
2025-12-18,25815.550781,25902.349609,25726.300781,25764.699219,197600
2025-12-19,25966.400391,25993.349609,25880.449219,25911.500000,382900
2025-12-22,26172.400391,26180.699219,26047.800781,26055.849609,253000
2025-12-23,26177.150391,26233.550781,26119.050781,26205.199219,216600
2025-12-24,26142.099609,26236.400391,26123.000000,26170.650391,188800


In [80]:
# -----------------------------
# MACD INDICATOR
# -----------------------------
df = nifty_data
macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)

df['MACD'] = macd['MACD_12_26_9']
df['MACD_Signal'] = macd['MACDs_12_26_9']
df['MACD_Hist'] = macd['MACDh_12_26_9']

# -----------------------------
# MACD CROSSOVER SIGNALS
# -----------------------------
df['Signal'] = 0

# BUY: MACD crosses above Signal
df.loc[
    (df['MACD'] > df['MACD_Signal']) &
    (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)),
    'Signal'
] = 1

# SELL: MACD crosses below Signal
df.loc[
    (df['MACD'] < df['MACD_Signal']) &
    (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)),
    'Signal'
] = -1

# -----------------------------
# LABEL SIGNALS
# -----------------------------
df['Trade'] = df['Signal'].map({1: 'BUY', -1: 'SELL', 0: 'HOLD'})

# Trade only when histogram strength confirms
df.loc[(df['Signal'] == 1) & (df['MACD_Hist'] < 0), 'Signal'] = 0
df.loc[(df['Signal'] == -1) & (df['MACD_Hist'] > 0), 'Signal'] = 0


In [81]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ensure index is datetime
df.index = pd.to_datetime(df.index)

# -----------------------------
# CREATE SUBPLOTS
# -----------------------------
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    row_heights=[0.65, 0.35],
    subplot_titles=("NIFTY Price with Buy/Sell Signals", "MACD Indicator")
)

# -----------------------------
# PRICE LINE
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['Close'],
        mode='lines',
        name='Close Price',
        line=dict(width=2)
    ),
    row=1, col=1
)

# -----------------------------
# BUY SIGNALS
# -----------------------------
buy_df = df[df['Signal'] == 1]

fig.add_trace(
    go.Scatter(
        x=buy_df.index,
        y=buy_df['Close'],
        mode='markers',
        name='BUY',
        marker=dict(
            symbol='triangle-up',
            size=12,
            color='green'
        )
    ),
    row=1, col=1
)

# -----------------------------
# SELL SIGNALS
# -----------------------------
sell_df = df[df['Signal'] == -1]

fig.add_trace(
    go.Scatter(
        x=sell_df.index,
        y=sell_df['Close'],
        mode='markers',
        name='SELL',
        marker=dict(
            symbol='triangle-down',
            size=12,
            color='red'
        )
    ),
    row=1, col=1
)

# -----------------------------
# MACD LINE
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD'],
        name='MACD',
        line=dict(color='blue', width=2)
    ),
    row=2, col=1
)

# -----------------------------
# SIGNAL LINE
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD_Signal'],
        name='Signal Line',
        line=dict(color='orange', width=2)
    ),
    row=2, col=1
)

# -----------------------------
# HISTOGRAM
# -----------------------------
fig.add_trace(
    go.Bar(
        x=df.index,
        y=df['MACD_Hist'],
        name='Histogram',
        marker_color=df['MACD_Hist'].apply(
            lambda x: 'green' if x >= 0 else 'red'
        )
    ),
    row=2, col=1
)

# -----------------------------
# ZERO LINE FOR MACD
# -----------------------------
fig.add_hline(
    y=0,
    line_width=1,
    line_dash="dash",
    line_color="gray",
    row=2, col=1
)

# -----------------------------
# LAYOUT
# -----------------------------
fig.update_layout(
    height=800,
    title="NIFTY MACD Strategy (Buy/Sell Signals)",
    showlegend=True,
    xaxis_rangeslider_visible=False,
    template="plotly_white"
)

fig.show()


In [ ]:
df['StrongSignal'] = 0

# -------- STRONG BUY --------
df.loc[
    (df['MACD'] > df['MACD_Signal']) &
    (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)) &
    (df['MACD'] > 0.75),
    'StrongSignal'
] = 1

# -------- STRONG SELL --------
df.loc[
    (df['MACD'] < df['MACD_Signal']) &
    (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)) &
    (df['MACD'] < -0.75),
    'StrongSignal'
] = -1

df['TradeType'] = df['StrongSignal'].map({
    1: 'STRONG BUY',
    -1: 'STRONG SELL',
    0: 'NO TRADE'
})


In [83]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.65, 0.35],
    vertical_spacing=0.07,
    subplot_titles=(
        "NIFTY Price – STRONG MACD Signals Only",
        "MACD (Filtered: |MACD| > 0.5)"
    )
)

# ---------------- PRICE ----------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['Close'],
        name="Close",
        line=dict(width=2)
    ),
    row=1, col=1
)

# ---------------- STRONG BUY ----------------
sb = df[df['StrongSignal'] == 1]

fig.add_trace(
    go.Scatter(
        x=sb.index,
        y=sb['Close'],
        mode='markers',
        name="STRONG BUY",
        marker=dict(
            symbol='triangle-up',
            size=16,
            color='green'
        )
    ),
    row=1, col=1
)

# ---------------- STRONG SELL ----------------
ss = df[df['StrongSignal'] == -1]

fig.add_trace(
    go.Scatter(
        x=ss.index,
        y=ss['Close'],
        mode='markers',
        name="STRONG SELL",
        marker=dict(
            symbol='triangle-down',
            size=16,
            color='red'
        )
    ),
    row=1, col=1
)

# ---------------- MACD ----------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD'],
        name='MACD',
        line=dict(width=2)
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD_Signal'],
        name='Signal',
        line=dict(width=2)
    ),
    row=2, col=1
)

# ---------------- HISTOGRAM ----------------
fig.add_trace(
    go.Bar(
        x=df.index,
        y=df['MACD_Hist'],
        name='Histogram',
        marker_color=df['MACD_Hist'].apply(
            lambda x: 'green' if x > 0 else 'red'
        )
    ),
    row=2, col=1
)

# ---------------- THRESHOLD LINES ----------------
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2)
fig.add_hline(y=0.5, line_dash="dot", line_color="green", row=2)
fig.add_hline(y=-0.5, line_dash="dot", line_color="red", row=2)

# ---------------- LAYOUT ----------------
fig.update_layout(
    height=820,
    title="NIFTY – STRONG MACD Strategy (Noise Filtered)",
    template="plotly_white",
    xaxis_rangeslider_visible=False
)

fig.show()


In [84]:
# MACD slope (1-bar)
df['MACD_Slope'] = df['MACD'] - df['MACD'].shift(1)


In [85]:
df['StrongSignal'] = 0

# -------- STRONG BUY --------
df.loc[
    (df['MACD'] > df['MACD_Signal']) &
    (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)) &
    (df['MACD'] > 0.5) &
    (df['MACD_Slope'] > 0),     # 🔥 MACD must be rising
    'StrongSignal'
] = 1

# -------- STRONG SELL --------
df.loc[
    (df['MACD'] < df['MACD_Signal']) &
    (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)) &
    (df['MACD'] < -0.5) &
    (df['MACD_Slope'] < 0),     # 🔥 MACD must be falling
    'StrongSignal'
] = -1

# 2-bar MACD slope
df['MACD_Slope_2'] = df['MACD'] - df['MACD'].shift(2)

# ---- STRONG BUY: invalidate if slope not positive ----
df.loc[
    (df['StrongSignal'] == 1) &
    (df['MACD_Slope_2'] <= 0),
    'StrongSignal'
] = 0

# ---- STRONG SELL: invalidate if slope not negative ----
df.loc[
    (df['StrongSignal'] == -1) &
    (df['MACD_Slope_2'] >= 0),
    'StrongSignal'
] = 0




In [86]:
df['StrongSignal'] = 0

# STRONG BUY
df.loc[
    (df['MACD'] > df['MACD_Signal']) &
    (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)) &
    (df['MACD'] > 0.5) &
    (df['MACD'] - df['MACD'].shift(2) > 0),
    'StrongSignal'
] = 1

# STRONG SELL
df.loc[
    (df['MACD'] < df['MACD_Signal']) &
    (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)) &
    (df['MACD'] < -0.5) &
    (df['MACD'] - df['MACD'].shift(2) < 0),
    'StrongSignal'
] = -1


In [87]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df.index = pd.to_datetime(df.index)

# -----------------------------
# FILTER STRONG SIGNALS
# -----------------------------
buy_df = df[df['StrongSignal'] == 1]
sell_df = df[df['StrongSignal'] == -1]

# -----------------------------
# SUBPLOTS
# -----------------------------
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.65, 0.35],
    vertical_spacing=0.06,
    subplot_titles=(
        "NIFTY Price – STRONG MACD Buy / Sell Signals",
        "MACD (Strength & Direction Filtered)"
    )
)

# -----------------------------
# PRICE LINE
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['Close'],
        name="Close",
        line=dict(width=2)
    ),
    row=1, col=1
)

# -----------------------------
# STRONG BUY ARROWS
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=buy_df.index,
        y=buy_df['Close'],
        mode="markers",
        name="STRONG BUY",
        marker=dict(
            symbol="triangle-up",
            size=16,
            color="green"
        )
    ),
    row=1, col=1
)

# -----------------------------
# STRONG SELL ARROWS
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=sell_df.index,
        y=sell_df['Close'],
        mode="markers",
        name="STRONG SELL",
        marker=dict(
            symbol="triangle-down",
            size=16,
            color="red"
        )
    ),
    row=1, col=1
)

# -----------------------------
# MACD LINE
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD'],
        name="MACD",
        line=dict(width=2)
    ),
    row=2, col=1
)

# -----------------------------
# SIGNAL LINE
# -----------------------------
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD_Signal'],
        name="Signal",
        line=dict(width=2)
    ),
    row=2, col=1
)

# -----------------------------
# HISTOGRAM
# -----------------------------
fig.add_trace(
    go.Bar(
        x=df.index,
        y=df['MACD_Hist'],
        name="Histogram",
        marker_color=[
            "green" if v >= 0 else "red" for v in df['MACD_Hist']
        ]
    ),
    row=2, col=1
)

# -----------------------------
# MACD LEVEL LINES
# -----------------------------
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2)
fig.add_hline(y=0.5, line_dash="dot", line_color="green", row=2)
fig.add_hline(y=-0.5, line_dash="dot", line_color="red", row=2)

# -----------------------------
# LAYOUT
# -----------------------------
fig.update_layout(
    height=850,
    title="NIFTY – STRONG MACD Strategy (Momentum + Strength Filter)",
    template="plotly_white",
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02)
)

fig.show()


In [88]:
# Histogram slope (momentum expansion)
df['HIST_Slope'] = df['MACD_Hist'] - df['MACD_Hist'].shift(1)


df['StrongSignal'] = 0

# -------- STRONG BUY --------
df.loc[
    (df['MACD'] > df['MACD_Signal']) &
    (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)) &
    (df['MACD'] > 0.5) &
    (df['MACD'] - df['MACD'].shift(2) > 0) &        # MACD rising
    (df['MACD_Hist'] > 0.2) &                       # Strong histogram
    (df['HIST_Slope'] > 0),                         # Histogram expanding
    'StrongSignal'
] = 1

# -------- STRONG SELL --------
df.loc[
    (df['MACD'] < df['MACD_Signal']) &
    (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)) &
    (df['MACD'] < -0.5) &
    (df['MACD'] - df['MACD'].shift(2) < 0) &        # MACD falling
    (df['MACD_Hist'] < -0.2) &                      # Strong bearish hist
    (df['HIST_Slope'] < 0),                         # Momentum expanding
    'StrongSignal'
] = -1


fig.add_hline(y=0.2, line_dash="dot", line_color="green", row=2)
fig.add_hline(y=-0.2, line_dash="dot", line_color="red", row=2)
